In [1]:
import pandas as pd
combined_dataset = pd.read_pickle('./combined_dataset_notes.pkl')

In [2]:
combined_dataset.head()

,text,HADM_ID,ADMISSION_DAYS,GENDER,AGE,50801,50802,50803,50804,50805,...,G11,G12,G13,G14,G15,G16,G17,G18,G19,G20
47366,"[chest, pa, lat, clip, clip, number, radiology...",100001,7,0,35,462.0,0.0,24.0,26.0,1.0,...,0,1,0,0,0,0,0,0,0,1
47024,"[pm, liver, gallbladder, u, single, organ, cli...",100003,5,1,60,462.0,-6.0,24.0,17.0,1.0,...,0,1,0,0,0,1,0,0,0,0
9046,"[chest, portable, ap, clip, clip, number, radi...",100006,13,0,49,462.0,0.0,24.0,26.0,1.0,...,0,0,0,0,0,1,0,0,0,1
843,"[pm, chest, pa, lat, clip, clip, number, radio...",100009,5,1,61,462.0,2.0,24.0,28.0,1.0,...,0,0,0,0,0,0,0,1,0,1
43466,"[pm, chest, portable, ap, different, physician...",100010,5,0,54,462.0,0.0,23.0,25.0,1.0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
!pip install --upgrade gensim
!pip install -U gensim

Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)
Requirement already up-to-date: gensim in /home/akshara/anaconda3/lib/python3.7/site-packages (3.8.1)


In [4]:

from gensim.models import Word2Vec

In [5]:
#pass text here..
model = Word2Vec(combined_dataset.text,size=1500, window=10, min_count=3, workers=50)#play around with param
# summarize the loaded model
print(model)
# summarize vocabulary
words = list(model.wv.vocab)
print(words)
# access vector for one word
#print(model['sentence'])
# save model
model.save('model1500-10-3.bin')
# load model
new_model = Word2Vec.load('model1500-10-3.bin')
print(new_model)

Word2Vec(vocab=32438, size=1500, alpha=0.025)
['chest', 'pa', 'lat', 'clip', 'number', 'radiology', 'reason', 'eval', 'infiltrate', 'hospital', 'medical', 'condition', 'year', 'old', 'woman', 'vomiting', 'examination', 'final', 'report', 'radiograph', 'indication', 'evaluation', 'infiltrates', 'comparison', 'finding', 'compared', 'previous', 'relevant', 'change', 'evidence', 'pneumonia', 'pathological', 'abnormality', 'pleural', 'effusion', 'pulmonary', 'edema', 'normal', 'size', 'cardiac', 'silhouette', 'pm', 'liver', 'gallbladder', 'u', 'single', 'organ', 'portal', 'vein', 'thrombosis', 'tapable', 'ascites', 'please', 'mark', 'man', 'disease', 'hypertension', 'worsening', 'lfts', 'wet', 'read', 'gwp', 'fri', 'pv', 'patent', 'hepatopetal', 'flow', 'query', 'lliver', 'ultrasound', 'doctor', 'last', 'name', 'scale', 'color', 'doppler', 'son', 'ni', 'image', 'demonstrate', 'persistently', 'coarsened', 'echotexture', 'nodular', 'surface', 'consistent', 'known', 'history', 'cirrhosis', 'la

Word2Vec(vocab=32438, size=1500, alpha=0.025)


In [6]:
y = combined_dataset.iloc[:,485:]

In [7]:
y.columns

Index(['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9', 'G10', 'G11',
       'G12', 'G13', 'G14', 'G15', 'G16', 'G17', 'G18', 'G19', 'G20'],
      dtype='object')

In [8]:

x = combined_dataset.iloc[:,:485]
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.15)

In [9]:
'''%matplotlib inline
from sklearn.decomposition import PCA
from matplotlib import pyplot
X = model[model.wv.vocab]
pca = PCA(n_components=2)
result = pca.fit_transform(X)
# create a scatter plot of the projection
pyplot.scatter(result[:, 0], result[:, 1])
words = list(model.wv.vocab)
for i, word in enumerate(words):
    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
pyplot.show()'''

'%matplotlib inline\nfrom sklearn.decomposition import PCA\nfrom matplotlib import pyplot\nX = model[model.wv.vocab]\npca = PCA(n_components=2)\nresult = pca.fit_transform(X)\n# create a scatter plot of the projection\npyplot.scatter(result[:, 0], result[:, 1])\nwords = list(model.wv.vocab)\nfor i, word in enumerate(words):\n    pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))\npyplot.show()'

In [10]:
#voila

In [11]:
#avg_vec.shape

In [12]:
WordVectorz=dict(zip(model.wv.index2word,model.wv.vectors))
class AverageEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim =100 # because we use 100 embedding points 

    def fit(self, X, y):
        return self

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
AverageEmbeddingVectorizer(WordVectorz)

In [38]:
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.model_selection import GridSearchCV,StratifiedKFold
from sklearn.metrics import jaccard_score,roc_auc_score,confusion_matrix,hamming_loss

#clf_multilabel = OneVsRestClassifier(LGBMClassifier(iterations=1000,
#                                                                     learning_rate=0.15))

#from sklearn.feature_extraction.text import TfidfVectorizer

In [39]:
import numpy as np
pipe1=Pipeline([("wordVectz",AverageEmbeddingVectorizer(WordVectorz)),("multilabel",OneVsRestClassifier(XGBClassifier(iterations=1000,learning_rate=0.15,n_jobs=-1)))])
pipe1.fit(x_train.text,y_train)
# clf_multilabel.fit(x_train.text,y_train)#have to now oopsitsk


Pipeline(memory=None,
         steps=[('wordVectz',
                 <__main__.AverageEmbeddingVectorizer object at 0x7f985aee2590>),
                ('multilabel',
                 OneVsRestClassifier(estimator=XGBClassifier(base_score=0.5,
                                                             booster='gbtree',
                                                             colsample_bylevel=1,
                                                             colsample_bynode=1,
                                                             colsample_bytree=1,
                                                             gamma=0,
                                                             iterations=1000,
                                                             learning_rate=0.15,
                                                             max_delta_step=0,
                                                             max_depth=3,
                                                     

In [40]:
predicted1=pipe1.predict(x_test.text)

In [41]:
predicted1_prob_train = pipe1.predict_proba(x_train.text)
predicted1_prob_test = pipe1.predict_proba(x_test.text)

In [42]:
print(jaccard_score(y_test,predicted1,average='micro'))
print(jaccard_score(y_test,predicted1,average='macro'))
print(roc_auc_score(y_test,predicted1))
print(hamming_loss(y_test,predicted1))


0.5058436018016945
0.3771273552896256
0.6573500885474709
0.1823400205008054


In [43]:
predicted1

array([[0, 0, 0, ..., 1, 1, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 1, 0],
       ...,
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 1, 1, 1],
       [0, 0, 1, ..., 0, 0, 0]])

In [44]:
#pipe1.fit(x_train.text,y_train)
predicted1_prob_testdf = pd.DataFrame(predicted1_prob_test)
predicted1_prob_traindf = pd.DataFrame(predicted1_prob_train)

In [45]:
predicted1_prob_test.shape

(6829, 20)

In [46]:
predicted1_prob_testdf

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.006364,0.014811,0.381461,0.244952,0.226117,0.133170,0.422299,0.114607,0.154242,0.135912,0.000083,0.029295,0.206063,0.008920,0.000142,0.144964,0.027488,0.984861,0.983045,0.347243
1,0.060448,0.020596,0.792334,0.469381,0.554800,0.337727,0.471109,0.146749,0.755750,0.457869,0.001473,0.103819,0.162318,0.017451,0.000144,0.527361,0.014730,0.352813,0.349358,0.289949
2,0.011747,0.047712,0.474600,0.111793,0.401464,0.651509,0.855404,0.151393,0.127526,0.281706,0.000056,0.029521,0.174612,0.006954,0.000131,0.310654,0.001712,0.515857,0.806501,0.454475
3,0.009529,0.030799,0.778939,0.136471,0.182365,0.104084,0.989231,0.141261,0.191664,0.090097,0.000048,0.020778,0.120590,0.015656,0.000347,0.082227,0.001190,0.168287,0.069491,0.452310
4,0.155675,0.350358,0.850952,0.722703,0.364650,0.714532,0.905861,0.852393,0.701567,0.771462,0.001481,0.397246,0.220130,0.035454,0.000101,0.672055,0.041518,0.754205,0.359311,0.675329
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6824,0.095784,0.137530,0.860846,0.712012,0.310655,0.738019,0.927311,0.315240,0.471653,0.871125,0.000228,0.494851,0.312031,0.008207,0.000102,0.528199,0.029597,0.859444,0.631167,0.771659
6825,0.070175,0.068985,0.846435,0.632877,0.799036,0.569135,0.780485,0.465946,0.804712,0.705267,0.001412,0.456766,0.196635,0.019320,0.000352,0.601936,0.041650,0.657041,0.343640,0.647979
6826,0.042937,0.040011,0.640910,0.506048,0.862476,0.538513,0.452046,0.583061,0.328257,0.358777,0.000726,0.209272,0.294012,0.011878,0.000791,0.636439,0.031056,0.493248,0.481291,0.412246
6827,0.105662,0.080924,0.846555,0.791480,0.317569,0.542417,0.959383,0.258478,0.417617,0.915276,0.000217,0.367771,0.196251,0.009987,0.000130,0.452880,0.026391,0.855679,0.562148,0.643258


In [47]:
x_train.text.to_pickle('./train-nlp.pkl')
x_test.text.to_pickle('./test-nlp.pkl')
y_train.to_pickle('./y_train-nlp.pkl')
y_test.to_pickle('./y_test-nlp.pkl')
predicted1_prob_testdf.to_pickle('./pred-test-xgb.pkl')
predicted1_prob_traindf.to_pickle('./pred-train-xgb.pkl')